In [1]:
#Default
import pandas as pd
pd.options.display.float_format = '{:,}'.format
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
def reset(df):
    cols = df.columns
    return df.reset_index()[cols]
def print_counts(df):
    cols = df.columns
    for each in cols:
        print(each)
        print(df[each].value_counts())
        print('______________________________________')
        
def flatten(list_1):
    list_2 = []
    for each in list(list_1):
        try:
            for each_2 in each:
                list_2.append(each_2)
        except:
            pass
    return list_2
        
# ~
#Default Ending

import os
from tqdm import tqdm

print('OK !')

OK !


In [2]:
from haystack.utils import clean_wiki_text, convert_files_to_dicts, fetch_archive_from_http, print_answers
from haystack.nodes import DensePassageRetriever, EmbeddingRetriever, FARMReader, EntityExtractor
from haystack.document_stores import ElasticsearchDocumentStore
from haystack.pipelines import ExtractiveQAPipeline, DocumentSearchPipeline

from flask import Flask, request, jsonify

print('OK !')

INFO - haystack.modeling.model.optimization -  apex not found, won't use it. See https://nvidia.github.io/apex/


OK !


# Set similarity_type

In [3]:
# For DensePassageRetriever
similarity_type = "dot_product"

# For EmbeddingRetriever
# similarity_type = "cosine"

In [4]:
document_store_100 = ElasticsearchDocumentStore(host="localhost", port = "9200", index="document_100",
                                           similarity=similarity_type)
document_store_200 = ElasticsearchDocumentStore(host="localhost", port = "9200", index="document_200",
                                           similarity=similarity_type)
document_store_300 = ElasticsearchDocumentStore(host="localhost", port = "9200", index="document_300",
                                           similarity=similarity_type)

# Set document_store 

In [5]:
document_store = document_store_200


# Set Retriever

In [6]:
retriever = DensePassageRetriever(
    document_store=document_store,
    query_embedding_model="DPR-retriever/query_encoder",
    passage_embedding_model="DPR-retriever/passage_encoder",
    max_seq_len_query=64,
    max_seq_len_passage=256,
    batch_size=16,
    use_gpu=True,
    embed_title=True,
    use_fast_tokenizers=True,
)

INFO - haystack.modeling.utils -  Using devices: CPU
INFO - haystack.modeling.utils -  Number of GPUs: 0
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Model found locally at DPR-retriever/query_encoder
INFO - haystack.modeling.model.language_model -  Loaded DPR-retriever/query_encoder
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Model found locally at DPR-retriever/passage_encoder
INFO - haystack.modeling.model.language_model -  Loaded DPR-retriever/passage_encoder


# Set Reader

In [7]:
# reader = FARMReader(model_name_or_path="albert_xxlargev1_squad2_512-reader", use_gpu=True,
#                    context_window_size = 500)

# reader = FARMReader(model_name_or_path="roberta-base-squad2-reader", use_gpu=True,
#                    context_window_size = 500)

reader = FARMReader(model_name_or_path="minilm-uncased-squad2-reader", use_gpu=True)

INFO - haystack.modeling.utils -  Using devices: CPU
INFO - haystack.modeling.utils -  Number of GPUs: 0
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Model found locally at minilm-uncased-squad2-reader
INFO - haystack.modeling.model.language_model -  Loaded minilm-uncased-squad2-reader
INFO - haystack.modeling.model.adaptive_model -  Found files for loading 1 prediction heads
WARNING - haystack.modeling.model.prediction_head -  Some unused parameters are passed to the QuestionAnsweringHead. Might not be a problem. Params: {"training": false, "num_labels": 2, "ph_output_type": "per_token_squad", "model_type": "span_classification", "label_tensor_name": "question_answering_label_ids", "label_list": ["start_token", "end_token"], "metric": "squad", "name": "QuestionAnsweringHead"}
INFO - haystack.modeling.model.prediction_head -  Loading prediction head from min

# Set Pipeline

In [8]:
retriever_pipe = DocumentSearchPipeline(retriever)
reader_pipe = ExtractiveQAPipeline(reader,retriever)

# Set Meta Filter Dict

In [9]:
meta_list = list(document_store)
meta_list = [each.__dict__['meta'] for each in meta_list]
meta_df = pd.DataFrame(meta_list)

# By Sentence
main_col_list = ['central_bank', 'keyword']

# Editing this will depend on NER models
alt_col_list = ['ORG', 'LOC', 'PER', 'MISC']

fil_col_list = main_col_list + alt_col_list
min_count = 3
meta_fil_dict = {}
for col in fil_col_list:
    if col in alt_col_list and col in list(meta_df.columns):
        temp_df = pd.DataFrame(flatten(list(meta_df[col].values)))[0].value_counts().reset_index()
    elif col in main_col_list and col in list(meta_df.columns):
        temp_df = pd.DataFrame(list(meta_df[col].values))[0].value_counts().reset_index()
    
    if col in list(meta_df.columns):
        temp_df = temp_df[temp_df[0] >= min_count]
        temp_dict = dict(zip(temp_df['index'], temp_df[0]))
        meta_fil_dict[col] = temp_dict
        
return_meta_fil_dict = {}
return_meta_fil_dict['sentence'] = meta_fil_dict

# By Document files
meta_df = meta_df.groupby('file_name').first().reset_index()
min_count = 0
fil_col_list = ['central_bank', 'keyword']
meta_fil_dict = {}
for col in fil_col_list:
    if col in alt_col_list and col in list(meta_df.columns):
        temp_df = pd.DataFrame(flatten(list(meta_df[col].values)))[0].value_counts().reset_index()
    elif col in main_col_list and col in list(meta_df.columns):
        temp_df = pd.DataFrame(list(meta_df[col].values))[0].value_counts().reset_index()
    
    if col in list(meta_df.columns):
        temp_df = temp_df[temp_df[0] >= min_count]
        temp_dict = dict(zip(temp_df['index'], temp_df[0]))
        meta_fil_dict[col] = temp_dict
        
return_meta_fil_dict['document'] = meta_fil_dict

# Set API Server

In [10]:
app = Flask(__name__)

@app.route('/haystack_200_reader_pipe', methods=['POST'])
def haystack_200_reader_pipe():
    query_list = []
    if request.method == 'POST':
        body = request.get_json()        
        prediction = reader_pipe.run(
            query=body['query'], params={"Retriever": {"top_k": int(body['top_k_retriever'])},
                                         "Reader": {'top_k': int(body['top_k_reader'])},
                                         "filters": body['filters'],
                                         }
        )
        ans_dict = prediction['answers']
        ans_dict = [each.__dict__ for each in ans_dict]
        prediction['answers'] = ans_dict
        doc_dict = prediction['documents']
        doc_dict = [each.__dict__ for each in doc_dict]
        prediction['documents'] = doc_dict
        
        return prediction, 201
    
@app.route('/haystack_200_retriever_pipe', methods=['POST'])
def haystack_200_retriever_pipe():
    query_list = []
    if request.method == 'POST':
        body = request.get_json()        
        prediction = retriever_pipe.run(
            query=body['query'], params={"Retriever": {"top_k": int(body['top_k_retriever'])},
                                         "filters": body['filters'],
                                         }
        )
        doc_list = [each.__dict__ for each in prediction['documents']]
        prediction['documents'] = doc_list
        return prediction, 201
    
@app.route('/haystack_200_meta', methods=['GET'])
def haystack_200_meta():
    query_list = []
    if request.method == 'GET':        
        return return_meta_fil_dict, 201
    
print('OK !')

OK !


In [ ]:
app.run(port = 5201)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO - werkzeug -   * Running on http://127.0.0.1:5201/ (Press CTRL+C to quit)
INFO - haystack.modeling.model.optimization -  apex not found, won't use it. See https://nvidia.github.io/apex/
INFO - haystack.modeling.model.optimization -  apex not found, won't use it. See https://nvidia.github.io/apex/
INFO - haystack.modeling.model.optimization -  apex not found, won't use it. See https://nvidia.github.io/apex/
INFO - haystack.modeling.model.optimization -  apex not found, won't use it. See https://nvidia.github.io/apex/
INFO - haystack.modeling.model.optimization -  apex not found, won't use it. See https://nvidia.github.io/apex/
INFO - haystack.modeling.model.optimization -  apex not found, won't use it. See https://nvidia.github.io/apex/


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Inferencing Samples:   0%|                        | 0/1 [00:00<?, ? Batches/s]/opt/anaconda3/envs/py38hay/lib/python3.8/site-packages/haystack/modeling/model/prediction_head.py:485: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  start_indices = flat_sorted_indices // max_seq_len
Inferencing Samples: 100%|████████████████| 1/1 [00:01<00:00,  1.33s/ Batches]
INFO - werkzeug -  127.0.0.1 - - [02/Apr/2022 14:37:19] "POST /haystack_200_reader_pipe HTTP/1.1" 201 -
INFO - werkzeug -  127.0.0.1 - - [02/Apr/2022 14:37:35] "POST /haystack_200_retriever_pipe HTTP/1.1" 201 -
INFO - werkzeug -  127.0.0.1 - - [02/Apr/2022 14:37:40] "POST /haystack_200_retriever_pipe HTTP/1.1

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Inferencing Samples: 100%|████████████████| 1/1 [00:00<00:00,  1.49 Batches/s]
INFO - werkzeug -  127.0.0.1 - - [02/Apr/2022 14:46:32] "POST /haystack_200_reader_pipe HTTP/1.1" 201 -
Inferencing Samples: 100%|████████████████| 1/1 [00:00<00:00,  2.83 Batches/s]
INFO - werkzeug -  127.0.0.1 - - [02/Apr/2022 14:47:11] "POST /haystack_200_reader_pipe HTTP/1.1" 201 -
INFO - werkzeug -  127.0.0.1 - - [02/Apr/2022 14:53:24] "POST /haystack_200_retriever_pipe HTTP/1.1" 201 -
INFO - werkzeug -  127.0.0.1 - - [02/Apr/2022 14:56:56] "POST /haystack_200_retriever_pipe HTTP/1.1" 201 -
INFO - werkzeug -  127.0.0.1 - - [02/Apr/2022 14:57:51] "POST /haystack_200_retriever_pipe HTTP/1.1" 201 -
INFO - werkzeug -  127.0.0.1 - - [02/Apr/2022 14:59:56] "POST /haystack_200_retriever_pipe HTTP/1.1" 201 -
INFO - werkzeug -  127.0.0.1 - - [02/Apr/2022 15:00:02] "POST /haystack_200_retriever_pipe HTTP/1.1" 201 -
INFO - werkzeug -  127.0.0.1 - - [02/Apr/2022 15:00:13] "POST /haystack_200_retriever_pipe HTTP/1.1